In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_hub.tools.tavily_research import TavilyToolSpec
from llama_parse import LlamaParse
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.agent import AgentRunner
from llama_index.core.embeddings import resolve_embed_model
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from prompts import assistant_prompt  
from helper_functions import (fetch_user_flight_information, 
                              search_flights, 
                              cancel_ticket, 
                              update_ticket_to_new_flight,
                              book_hotel,
                              cancel_hotel,
                              update_hotel,
                              search_hotels,
                              book_car_rental,
                              cancel_car_rental,
                              update_car_rental,
                              search_car_rentals,
                              book_excursion,
                              search_trip_recommendations,
                              update_excursion,
                              cancel_excursion
                            )
from dotenv import load_dotenv
import os
import ast
from create_database import db

load_dotenv()

llm = Ollama(base_url='http://localhost:11434', model="llama3", request_timeout=360)

parser = LlamaParse(result_type="markdown")

file_extractor = {".docx": parser}

documents = SimpleDirectoryReader("./data").load_data()

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

if os.path.exists('./storage'):
    storage_context = StorageContext.from_defaults(persist_dir='./storage')
    vector_index = load_index_from_storage(storage_context=storage_context)
else:    
    vector_index = VectorStoreIndex.from_documents(documents)    
    vector_index.storage_context.persist()

query_engine = vector_index.as_query_engine(llm=llm)

lookup_policy = QueryEngineTool(
                        query_engine=query_engine,
                        metadata=ToolMetadata(
                            description="""Consult the company policies to check whether certain options are permitted.
                                            Use this before making any flight changes performing other 'write' events.""",
                            name='policy_documentation'
                        )
                    )

tavily_tool = TavilyToolSpec(
    api_key='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',
)

tools = [
    tavily_tool.to_tool_list()[0],
    lookup_policy,
    fetch_user_flight_information,
    search_flights,
    update_ticket_to_new_flight,
    cancel_ticket,
    book_hotel,
    cancel_hotel,
    update_hotel,
    search_hotels,
    book_car_rental,
    cancel_car_rental,
    update_car_rental,
    search_car_rentals,
    book_excursion,
    search_trip_recommendations,
    update_excursion,
    cancel_excursion
]

tools_descs = [tol.metadata.name for tol in tools]

customer_agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, context=assistant_prompt)

conversation_history = []

def store_interaction(user_question, agent_response):
    conversation_history.append({"question": user_question, "response": agent_response})

def get_conversation_context():
    # Concatenate previous Q&A pairs into a single string
    context = ""
    for interaction in conversation_history:
        context += f"Q: {interaction['question']} A: {interaction['response']} "
    return context
    

In [2]:
while (prompt := input("Enter a prompt (q to quit): ")) != "q":
    retries = 0
    result = ''
    while retries < 3:
        try:
            context = get_conversation_context()
            full_prompt = context + f"Q: {prompt} A: "
    
            result = customer_agent.query(full_prompt)
            break
        except Exception as e:
            retries += 1
            
    store_interaction(prompt, result)
    
    if retries >= 3:
        print("Unable to process request, try again...")
        continue
    
    print(result)

Thought: The customer wants to know something, but they haven't specified what yet. I need to use a tool to help me answer their question.
Action: search
Action Input: {'query': 'What are the flight options from Tehran to Mashhad?', 'max_results': 6}
Observation: [Document(id_='779b4c90-07b9-4b5c-ad0e-902782cb2e7e', embedding=None, metadata={'url': 'https://www.google.com/travel/flights/flights-from-tehran-to-mashhad.html'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Use Google Flights to plan your next trip and find cheap one way or round trip flights from Tehran to Mashhad. Find the best flights fast, track prices, and book with confidence.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='7defa536-17d2-4f6e-ad9f-04eb29690ced', embedding=None, metadata={'url': 'https://www.wego.com/flights/thr/mhd/cheapest-flights-from-tehran-to

KeyboardInterrupt: 